In [ ]:
import numpy as np
import pandas as pd
import torch
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_final = pd.read_csv('/content/drive/MyDrive/Llama_Project/df_final.csv')
df_final.head()

,Unnamed: 0,text
0,0,Title: Iteration Counter for Mandelbrot Fracta...
1,1,Design Problem Statement:\n\nDesign a multiple...
2,2,Problem Statement:\n\nDesign a Data Memory mod...
3,3,Problem Statement:\n\nDesign and implement a h...
4,4,"Problem Statement:\n\nDesign a module named ""c..."


In [ ]:
!pip install -q transformers datasets evaluate accelerate
!pip install -q peft
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

Training

In [ ]:
from transformers import BitsAndBytesConfig
!pip install bitsandbytes

In [ ]:
#parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_double_nested_quant = False
lora_r = 16
lora_alpha = 16
lora_dropout = 0.1

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(load_in_4bit=use_4bit,
                                bnb_4bit_use_double_quant=use_double_nested_quant,
                                bnb_4bit_quant_type=bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype=compute_dtype)

In [ ]:
!pip install -q accelerate
!pip install bitsandbytes
!pip install sentencepiece
model_name = 'openlm-research/open_llama_3b'
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache=False)
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
dataset = Dataset.from_pandas(df_final)
def tokenize(input):
  return tokenizer(input['text'], truncation=True)

lm_dataset = dataset.map(tokenize, batched=True)

# block_size = 128
# def group_texts(inputs):
#     total_length = len(inputs[list(inputs.keys())[0]])
#     if total_length >= block_size:
#         total_length = (total_length // block_size) * block_size
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in inputs.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

# lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/7664 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
!pip install -q trl
from trl import SFTTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM

num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
warmup_ratio = 0.03
group_by_length = False
save_steps = 0
logging_steps = 25
disable_tqdm= False
output_dir = '\content\drive\MyDrive\LLM\llama-7b'
packing = True
max_seq_length = 1024
hf_model_repo = "llama-3b-hdl"

training_args = TrainingArguments(
 output_dir=output_dir,
 num_train_epochs=num_train_epochs,
 per_device_train_batch_size=per_device_train_batch_size,
 gradient_accumulation_steps=gradient_accumulation_steps,
 gradient_checkpointing=gradient_checkpointing,
 optim=optim,
 logging_steps=logging_steps,
 save_strategy="epoch",
 learning_rate=learning_rate,
 weight_decay=weight_decay,
 fp16=fp16,
 bf16=bf16,
 max_grad_norm=max_grad_norm,
 warmup_ratio=warmup_ratio,
 group_by_length=group_by_length,
 lr_scheduler_type=lr_scheduler_type,
 disable_tqdm=disable_tqdm,
 report_to="tensorboard",
 seed=224
)

peft_config = LoraConfig(lora_alpha=lora_alpha,
                         lora_dropout=lora_dropout,
                         r=lora_r,
                         bias="none",
                         task_type="CAUSAL_LM")

def format_instruction(sample):
	return f"""### Instruction:{sample}"""

trainer = SFTTrainer(model=model,
                     train_dataset=lm_dataset,
                     peft_config=peft_config,
                     tokenizer=tokenizer,
                     max_seq_length=max_seq_length,
                     packing=packing,
                     args=training_args,
                     formatting_func=format_instruction)

In [ ]:
trainer.train()
#model.push_to_hub("llama-trained")

Token indices sequence length is longer than the specified maximum sequence length for this model (9169 > 2048). Running this sequence through the model will result in indexing errors
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,0.672800
50,0.733900
75,0.647500
100,0.586300
125,0.555100
150,0.580100
175,0.518100
200,0.495700
225,0.517600
250,0.489800


Model training failed after training for 5 hrs 43 mins due to runtime expairy. It will need to be trained an additional 7 hours to complete the training

Merging base model with adapter weights

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
 args.output_dir,
 low_cpu_mem_usage=True,
 return_dict=True,
 torch_dtype=torch.float16,
)

merged_model = model.merge_and_unload()

merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

Inference Testing

In [ ]:
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,)
prompt = " "
input = tokenizer(prompt, return_tensor="pt", truncation=True).input_ids
output = model.generate(input_ids=input, max_new_tokens=1000, do_sample=True, top_p=0.9,temperature=0.9)

In [ ]:
# from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# !pip install -q -U accelerate
# !pip install -q -U transformers

In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(output_dir='/content/drive/MyDrive/Llama_Project/gpt2-large-hdl',
#                                   per_device_train_batch_size=1,
#                                   evaluation_strategy="epoch",
#                                   learning_rate=2e-5,
#                                   weight_decay=0.01,
#                                   push_to_hub=True,
#                                   )

# trainer = Trainer(model=model,
#                   args=training_args,
#                   train_dataset=tokenized_dataset['train'],
#                   eval_dataset=tokenized_dataset['test'],
#                   data_collator=data_collator,
#                   )

In [ ]:
# torch.cuda.empty_cache()
# trainer.train()

In [ ]:
# model.push_to_hub('gpt2-large_hdl')
# tokenizer.push_to_hub('gpt2-large_hdl')